In [2]:
# Initialize the notebook
using Pkg
Pkg.instantiate()
Pkg.activate("..")
#include("../src/Particles.jl")
using Particles
using Plots
using NetCDF

nothing

  Activating project at `j:\Master_Thesis\particles.jl`


In [9]:
using NetCDF
d=NetCDF.open("locxx_map.nc")
@show d.vars
z= d["mesh2d_layer_z"][:]
@show z' # z is a 1D array from -10 to 0
ts = d["time"][:] # d["time"].atts
@show ts' # ts is a 1D array with the time steps in seconds

d.vars = Dict{String, NcVar}("mesh2d_ucx" => Disk Array with size 40 x 400 x 21
, "mesh2d_taus" => Disk Array with size 400 x 21
, "mesh2d_edge_nodes" => Disk Array with size 2 x 1201
, "mesh2d_u0" => Disk Array with size 40 x 1201 x 21
, "mesh2d_Numlimdt" => Disk Array with size 400 x 21
, "mesh2d_tureps1" => Disk Array with size 41 x 1201 x 21
, "mesh2d_q1" => Disk Array with size 40 x 1201 x 21
, "mesh2d_interface_z" => Disk Array with size 41
, "mesh2d_edge_y" => Disk Array with size 1201
, "mesh2d_face_y" => Disk Array with size 400
, "mesh2d_edge_faces" => Disk Array with size 2 x 1201
, "projected_coordinate_system" => Disk Array with size 
, "mesh2d_u1" => Disk Array with size 40 x 1201 x 21
, "mesh2d_s0" => Disk Array with size 400 x 21
, "time" => Disk Array with size 21
, "mesh2d_face_x" => Disk Array with size 400
, "mesh2d_node_x" => Disk Array with size 802
, "mesh2d_face_x_bnd" => Disk Array with size 4 x 400
, "mesh2d_face_nodes" => Disk Array with size 4 x 400
, "mesh2


z' = [-9.875 -9.625 -9.375 -9.125 -8.875 -8.625 -8.375 -8.125 -7.875 -7.625 -7.375 -7.125 -6.875 -6.625 -6.375 -6.125 -5.875 -5.625 -5.375 -5.125 -4.875 -4.625 -4.375 -4.125 -3.875 -3.625 -3.375 -3.125 -2.875 -2.625 -2.375 -2.125 -1.875 -1.625 -1.375 -1.125 -0.875 -0.625 -0.375 -0.125]


ts' = [0.0 30.00000000000005 59.99999999999979 89.9999999999995 119.99999999999922 149.99999999999997 180.0000000000011 210.00000000000225 240.00000000000338 270.0000000000045 300.0000000000056 330.00000000000676 360.0000000000079 390.00000000000904 420.0000000000102 450.0000000000113 480.00000000001245 510.0000000000136 540.0000000000094 570.0000000000049 600.0]


1×21 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  30.0  60.0  90.0  120.0  150.0  …  480.0  510.0  540.0  570.0  600.0

In [19]:
# open the netcdf file with dflow.jl
dflow_map=load_nc_info("data",r"locxx_map.nc")
t0=get_reftime(dflow_map)
itime=10
u_velocity=load_nc_map_slice(dflow_map,"mesh2d_ucx",itime)
v_velocity=load_nc_map_slice(dflow_map,"mesh2d_ucy",itime)
times = get_times(dflow_map,t0)
@show times' # 0 to 600 seconds

nothing

┌ Info: data\locxx_map.nc
└ @ Particles J:\Master_Thesis\particles.jl\src\dflow.jl:27


times' = Any[0.0 30.00000000000005 59.99999999999979 89.9999999999995 119.99999999999922 149.99999999999997 180.0000000000011 210.00000000000225 240.00000000000338 270.0000000000045 300.0000000000056 330.00000000000676 360.0000000000079 390.00000000000904 420.0000000000102 450.0000000000113 480.00000000001245 510.0000000000136 540.0000000000094 570.0000000000049 600.0]


In [20]:
# now with interpolation
interp=load_dflow_grid(dflow_map,50,false)
u1,v1=initialize_interpolation(dflow_map,interp,t0)

compute index:


- j:\Master_Thesis\particles.jl\case_lock_exchange\data\locxx_map.nc


initialize caching for j:\Master_Thesis\particles.jl\case_lock_exchange\data\locxx_map.nc mesh2d_ucx...


initialize caching for j:\Master_Thesis\particles.jl\case_lock_exchange\data\locxx_map.nc mesh2d_ucy...


(Particles.var"#f#20"{Interpolator, Float64, Symbol, Particles.var"#weights#19", Particles.var"#update_cache_backwards#17"{Vector{Any}}, Particles.var"#update_cache_forwards#14"{Vector{Int64}, Vector{Any}, Bool, Int64}, Vector{Any}}(Interpolator(Grid[Grid([299.25000000000006, 299.25000000000006, 298.50000000000006, 298.50000000000006, 297.75000000000006, 297.75000000000006, 297.00000000000006, 297.00000000000006, 296.25000000000006, 296.25000000000006  …  2.2500000000000004, 2.2500000000000004, 1.5000000000000002, 1.5000000000000002, 0.7500000000000001, 0.7500000000000001, 300.00000000000006, 300.00000000000006, 0.0, 0.0], [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0  …  0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0], [1 799 800 2; 3 1 2 4; … ; 797 795 796 798; 801 797 798 802], Int64[], [0.0, 300.00000000000006, 0.0, 1.0], [4, 4, 4, 4, 4, 4, 4, 4, 4, 4  …  4, 4, 4, 4, 4, 4, 4, 4, 4, 4], Grid[Grid([299.25000000000006, 299.25000000000006, 298.50000000000006, 298.50000000000006, 

In [21]:
# investigate the input
@show interp.grids[1].bbox' #boundng box in xy coordinates 

xs = interp.grids[1].xnodes
ys = interp.grids[1].ynodes

nothing

((interp.grids[1]).bbox)' = [0.0 300.00000000000006 0.0 1.0]


In [22]:
x_middle = 150.0
y_middle = 0.5
z_surface = 0.0
z_middle = -5.0
z_bottom = -10.0
t=20.0
u_surface = u1(x_middle,y_middle,z_surface,t) #x y z t
u_middle = u1(x_middle,y_middle,z_middle,t)
u_bottom = u1(x_middle,y_middle,z_bottom,t)
println("u = $(u_surface), $(u_middle), $(u_bottom) at z=$(z_surface), $(z_middle), $(z_bottom) at t=$(t) s")
# TODO PROBLEM: u is same at all depths --> probably all at the surface

u = -0.3334079305027215, -0.3334079305027215, -0.3334079305027215 at z=0.0, -5.0, -10.0 at t=20.0 s
